# QAM Modulator
Modulation is an important part of analog communication. Analog communication is needed to transfer information where we cannot send digital signals. That includes air, cables, optic fibers etc.

The purpose of a modulator is to convers one or more bits to a symbol.

There are many modulation techniques but QAM is probably the most popular. QAM is both an analog and a digital modulation scheme chat combines together PSK and ASK modulations. For more reading refer to Wikipedias "Quadrature amplitude modulation" article:
https://en.wikipedia.org/wiki/Quadrature_amplitude_modulation

My goal was to create a generic QAM modulator. I ended creating a modulator that can:
* Receive data in binary and in raw number form
* Modulate using binary and Gray code constellation
* Modulate any QAM where M=2^2k, k=1,2,3...

In [1]:
from numpy import log2, arange, sqrt, flip, concatenate, zeros, array, floor, sqrt, array, bitwise_xor
from numpy.random import randint

First lets create our constants.
* symbol_num: number of QAM symbols to create
* M: QAM form

In [2]:
symbol_num = 20
M = 64

sqrt_M = sqrt(M).astype(int)
k = log2(M).astype(int)

Lets generate some random data.

The modulator expects the data to be arranged where I is first followed by Q and each one of them represented in big-endian: https://en.wikipedia.org/wiki/Endianness

In [3]:
data_input = randint(2, size=k*symbol_num)
data_input

array([1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 1])

The next code will generate a vector that we will use to convert the the binnary constellation to Gray code constellation.

Inspiration for this code came from an article by Krishna Sankar: http://www.dsplog.com/2008/06/01/binary-to-gray-code-for-16qam/

*Notice the order of the numbers

In [4]:
vect = array(range(sqrt_M))
gary_constallation = bitwise_xor(vect, floor(vect/2).astype(int))
gary_constallation

array([0, 1, 3, 2, 6, 7, 5, 4], dtype=int32)

Next we will create a vector that will us convert the Gray code to actual QAM symbols.

In [5]:
vect = arange(1, sqrt(M), 2)
symbols = concatenate((flip(-vect, axis=0), vect)).astype(int)
symbols

array([-7, -5, -3, -1,  1,  3,  5,  7])

Now we reshape the data so that each row represents one QAM symbol

In [6]:
data_input = data_input.reshape((-1,k))
data_input

array([[1, 1, 0, 1, 1, 1],
       [0, 0, 1, 0, 1, 0],
       [0, 1, 1, 0, 1, 0],
       [1, 1, 1, 1, 1, 0],
       [0, 1, 1, 0, 0, 1],
       [0, 1, 0, 0, 0, 1],
       [1, 1, 0, 0, 0, 1],
       [1, 0, 1, 1, 1, 1],
       [0, 1, 0, 0, 1, 1],
       [0, 0, 1, 0, 0, 1],
       [1, 0, 0, 1, 1, 1],
       [0, 1, 1, 0, 0, 1],
       [0, 1, 1, 1, 1, 0],
       [1, 0, 0, 0, 1, 1],
       [1, 1, 1, 1, 0, 1],
       [1, 1, 0, 1, 1, 0],
       [1, 0, 1, 0, 1, 0],
       [1, 0, 1, 1, 0, 0],
       [0, 1, 1, 0, 0, 1],
       [0, 0, 0, 1, 1, 1]])

Finally we will convert the bits to Gray coded QAM symbols

In [7]:
I = zeros((data_input.shape[0],))
Q = zeros((data_input.shape[0],))
for n in range(int(data_input.shape[1] / 2)):
    I = I + data_input[:,n] * 2 ** n
for n in range(int(data_input.shape[1]/2),int(data_input.shape[1])):
    Q = Q + data_input[:,n] * 2 ** (n - int(data_input.shape[1]/2))

I = I.astype(int)
Q = Q.astype(int)

In [8]:
I = gary_constallation[I]
Q = gary_constallation[Q]
I = symbols[I]
Q = symbols[Q]

S = I + 1j * Q
S

array([-3.+1.j,  5.-1.j,  3.-1.j,  1.-3.j,  3.+5.j, -1.+5.j, -3.+5.j,
        7.+1.j, -1.+3.j,  5.+5.j, -5.+1.j,  3.+5.j,  3.-3.j, -5.+3.j,
        1.+7.j, -3.-3.j,  7.-1.j,  7.-5.j,  3.+5.j, -7.+1.j])

The QAM.py file was created based on the code above:

In [9]:
from QAM import modulator

Sn = modulator(data_input, M)
Sn

array([-3.+1.j,  5.-1.j,  3.-1.j,  1.-3.j,  3.+5.j, -1.+5.j, -3.+5.j,
        7.+1.j, -1.+3.j,  5.+5.j, -5.+1.j,  3.+5.j,  3.-3.j, -5.+3.j,
        1.+7.j, -3.-3.j,  7.-1.j,  7.-5.j,  3.+5.j, -7.+1.j])

In [10]:
S-Sn

array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j])